In [ ]:
from huggingface_hub import login

# 🔑 Autenticación en Hugging Face
login("")

# **Evaluación Phi-3.5-mini-instruct-awq - Intento 1**

In [ ]:
!pip install autoawq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00

In [ ]:
# 1. Importar librerías
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 2. Cargar dataset desde Excel
file_path = "/content/BenchmarkSpanish.xlsx"
df = pd.read_excel(file_path)

# 3. Configurar modelo cuantizado (versión para Transformers)
model_id = "thesven/Phi-3.5-mini-instruct-awq"

# 4. Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Detecta automáticamente la GPU
    trust_remote_code=True  # Por si es necesario para la arquitectura
)

# 5. Parámetros de generación (equivalente al pipeline original)
generation_config = {
    "do_sample": True,
    "temperature": 0.01,
    "top_p": 0.9,
    "repetition_penalty": 1.1,
    "max_new_tokens": 512
}

# 6. Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishPhiQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishPhiQ1.xlsx")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


🔸 Answer: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
 Vamos pensar paso a paso.[Proceso de Solución Textual]:

Paso 1: Entender la información dada para el Polinomio Cuadrático \( P(x) \) con un coeficiente principal positivo (\( c_p = 2 \)). El punto por el que pase es \( (16, 54) \), lo cual nos da una ecuación usando la forma general del polinomio cuadrático \( ax^2 + bx + c \):
\[ 2(16)^2 + 16b + c = 54 \]
Simplificado esto se convierte en:
\[ 512 + 16b + c = 54 \]
O sea:
\[ 16b + c = 54 - 512 \]
\[ 16b + c = -458 \] ... [Equación A]

Paso 2: Aplicar similarmente al Polinomio Cuadrático \( Q(x) \) con un coeficiente principal negativo (\( c_q = -2 \)), también cruzando otro punto \( (20, 53) \):
\[ (-2)(20)^2 + 20d + e = 53 \]
Esto simplifica como:
\[ -800 + 20d + e = 53 \]
Ordenándolo obtene

# **Intento 2**

In [ ]:
#Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishPhiQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishPhiQ2.xlsx")

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
🔸 Answer: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
 Vamos pensar paso a paso.[Proceso de Solución Textual]:

Paso 1: Entender la información dada para el Polinomio Cuadrático \( P(x) \) con un coeficiente principal positivo (\( c_p = 2 \)). El punto por el que pase es \( (16, 54) \), lo cual nos da una ecuación usando la forma general del polinomio cuadrático \( ax^2 + bx + c \):
\[ 2(16)^2 + 16b + c = 54 \]
Simplificado esto se convierte en:
\[ 512 + 16b + c = 54 \]
O sea:
\[ 16b + c = 54 - 512 \]
\[ 16b + c = -458 \] ... [Equación A]

Paso 2: Apli

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishPhiQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishQPhiQ3.xlsx")

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
🔸 Answer: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
 Vamos pensar paso a paso.[Proceso de Solución Textual]:

Paso 1: Entender la información dada para el Polinomio Cuadrático \( P(x) \) con un coeficiente principal positivo (\( c_p = 2 \)). El punto por el que pase es \( (16, 54) \), lo cual nos da una ecuación usando la forma estándar del polinomio cuadrático \( ax^2 + bx + c \):
\[ 2(16)^2 + 16b + c = 54 \]
Simplificado esto se convierte en:
\[ 512 + 16b + c = 54 \]
O sea:
\[ 16b + c = 54 - 512 \]
\[ 16b + c = -458 \] ... [Equación A]

Paso 2: Apl

# **Evaluación Inglés - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkEnglish.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishPhiQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishPhiQ1.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


🔸 Answer: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
 We know that a quadratic polynomial can be written in the form $ax^2+bx+c$. Since we are given the values of $a$, let's assume they take on standard forms: for $P(x)$, it is $2x^2+b_px+c_p$; for $Q(x)$, it is $-2x^2+b_qx+c_q$. 
We also know from point-to-point interpolation (using either method or graphing software), these equations satisfy: $$ \begin{cases} P(16)=54 \\ Q(16)=54\\ P(20)=53 \\ Q(20)=53\end{cases}$$ Substituting into our assumed equation gives us four linear systems to solve simultaneously; however this would require tedious calculations so instead I will use symmetry properties about parabolas with respect to their axis of symmetry which passes between any pair of corresponding vertices/intercepts as well as other methods such as completing squares etc., but ultima

# **Intento 2**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishPhiQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishPhiQ2.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
🔸 Answer: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
 We know that a quadratic polynomial can be written in the form $ax^2+bx+c$. Since we are given the values of $a$, let's assume they take on standard forms: for $P(x)$, it is $2x^2+b_px+c_p$; for $Q(x)$, it is $-2x^2+b_qx+c_q$. 
We also know from point-to-point interpolation (using either method or graphing software), these equations satisfy: $$ \begin{cases} P(16)=54 \\ Q(16)=54\\ P(20)=53 \\ Q(20)=53\end{cases}$$ Substituting into our assumed equation gives us four linear systems to solve simultaneously; however this would require tedious calculations so instead I will us

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishPhiQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishPhiQ3.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
🔸 Answer: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
 We know that a quadratic polynomial can be written in the form $ax^2+bx+c$. Since we are given the values of $a$, let's assume they take on standard forms: for $P(x)$, it is $2x^2+b_px+c_p$; for $Q(x)$, it is $-2x^2+b_qx+c_q$. 
We also know from point-to-point interpolation (using either method or graphing software), these equations satisfy: $$ \begin{cases} P(16)=54 \\ Q(16)=54\\ P(20)=53 \\ Q(20)=53\end{cases}$$ Substituting into our assumed equation gives us four linear systems to solve simultaneously; however this would require tedious calculations so instead I will us

# **Evaluación en Francés - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkFrench.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchPhiQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchPhiQ1.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
 Réfléchissons étape par étape.[Processus de solution textuelle]:

Étape 1: Comprendre la forme d'un polynôme quadratique général.
Un polynôme quadratique peut être écrit sous la forme \( ax^2 + bx + c \), où \( a \), \( b \) et \( c \) sont des constantes, avec \( a \) étant le coefficient du terme au carré. Dans notre cas, nous savons que \( a = 2 \) pour \( p(x) \) et \( a = -2 \) pour \( q(x) \).

Étape 2: Utiliser les informations données sur l'intersection à un point spécifique.
Nous connaissons deu

# **Intento 2**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchPhiQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchPhiQ2.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
 Réfléchissons étape par étape.[Processus de solution textuelle]:

Étape 1: Comprendre la forme d'un polynôme quadratique général.
Un polynôme quadratique peut être écrit sous la forme \( ax^2 + bx + c \), où \( a \), \( b \) et \( c \) sont des constantes, avec \( a \) étant le coefficient du terme au carré. Dans notre cas, nous savons que \( a = 2 \) pour \( p(x) \) et \( a = -2 \) pour \( q(x) \).

Étape 2: Utiliser les informations données sur l'intersection à un point spécifique.
Nous connaissons deu

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchPhiQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchPhiQ3.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
 Réfléchissons étape par étape.[Processus de solution textuelle]:

Étape 1: Comprendre la forme d'un polynôme quadratique général.
Un polynôme quadratique peut être écrit sous la forme \( ax^2 + bx + c \), où \( a \), \( b \) et \( c \) sont des constantes, avec \( a \) étant le coefficient du terme au carré. Dans ce problème, on nous dit que les coefficients principaux pour \( p(x) \) et \( q(x) \) sont 2 et -2, respectivement. Cela signifie que nos polynômes se ressemblent comme suit :
\[ p(x) = 2x^2 + 

# **Evaluación en Alemán - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkGerman.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanPhiQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanPhiQ1.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
 Denken wir Schritt für Schritt.[Textueller Lösungsprozess]: Um das gegebene mathematische Problem zu lösen, müssen wir folgende Schritte ausführen:

1. Verstehen der gegebenen Informationen: Wir haben zwei quadratische Polynome mit den Formeln \( P(x) = 2ax^2 + bx + c \) und \( Q(x) = -2ax^2 + dx + e \), wobei \( a \), \( b \), \( d \) und \( e \) Konstanten sind. Beide Polynome passieren die Punkte (16, 54) und (20, 53).

2. Aufstellen von Gleichungen basierend auf dem ersten Paar Koordinaten: Für jeden Polynomwert an einem dieser Punkte setzen wir eine entsprechende Gleichun

# **Intento 2**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanPhiQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanPhiQ2.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
 Denken wir Schritt für Schritt.[Textueller Lösungsprozess]: Um das gegebene mathematische Problem zu lösen, müssen wir folgende Schritte ausführen:

1. Verstehen der gegebenen Informationen: Wir haben zwei quadratische Polynome mit den Formeln \( P(x) = 2ax^2 + bx + c \) und \( Q(x) = -2ax^2 + dx + e \), wobei \( a \), \( b \), \( d \) und \( e \) Konstanten sind. Beide Polynome schneiden sich bei den Punkten (16, 54) und (20, 53).

2. Aufstellen von Gleichungen basierend auf dem ersten Paar Koordinaten: Anwendung dieser Punkte in beide Polynomgleichungen ergibt uns ein System

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanPhiQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanPhiQ3.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
 Denken wir Schritt für Schritt.[Textueller Lösungsprozess]: Um das gegebene mathematische Problem zu lösen, müssen wir folgende Schritte ausführen:

1. Verstehen der gegebenen Informationen: Wir haben zwei quadratische Polynome mit den Formeln \( P(x) = 2ax^2 + bx + c \) und \( Q(x) = -2ax^2 + dx + e \), wobei \( a \), \( b \), \( d \) und \( e \) Konstanten sind. Beide Polynome passieren die Punkte (16, 54) und (20, 53).

2. Aufstellen von Gleichungen basierend auf dem ersten Paar Koordinaten: Wenn beide Polynome diese Punkte berühren, können sie wie folgt geschrieben werden: